In [1]:
#!python
#cython: language_level=3

import cplex
import numpy as np

class Custom_Node():
    def __init__(self, name):
        self.name = name

class Custom_Edge():
    def __init__(self,node1,node2):
        self.node1 = node1
        self.node2 = node2
    def identify(self):
        print(f'Connecting {node1.name} to {node2.name}')
        

def vertex_cover(vertices, edges):
    prob = cplex.Cplex()
    prob.set_problem_name("Minimum Vertex Cover")
    prob.set_problem_type(cplex.Cplex.problem_type.LP)
    prob.objective.set_sense(prob.objective.sense.minimize)
    
    names = [vertice for vertice in vertices]

    # Objective (linear) weights
    w_obj = np.repeat(1, len(names)).tolist()
    # Lower bounds. Since these are all zero, we could simply not pass them in as
    # all zeroes is the default.
    low_bnd = np.repeat(0, len(names)).tolist()
    # Upper bounds. The default here would be cplex.infinity, or 1e+20.
    upr_bnd = np.repeat(1, len(names)).tolist()
    prob.variables.add(names=names, obj=w_obj, lb=low_bnd, ub=upr_bnd)
    all_int = [(var, prob.variables.type.integer) for var in names]
    prob.variables.set_types(all_int)
    
    constraints = []
    for edge in edges:
        constraints.append([[edge[0],edge[1]], [1,1]])
    print(constraints)
    # Constraint names
    constraint_names = ["".join(x[0]) for x in constraints]
    print(constraint_names)
    # Each edge must have at least one vertex
    rhs = [1] * len(constraints)
    print(rhs)
    # We need to enter the senses of the constraints. That is, we need to tell Cplex
    # whether each constrains should be treated as an upper-limit (≤, denoted "L"
    # for less-than), a lower limit (≥, denoted "G" for greater than) or an equality
    # (=, denoted "E" for equality)
    constraint_senses = ["G"] * len(constraints)

    # And add the constraints
    prob.linear_constraints.add(names=constraint_names,
                                lin_expr=constraints,
                                senses=constraint_senses,
                                rhs=rhs)
    # Solve the problem
    #print("Problem Type: %s" % prob.problem_type[prob.get_problem_type()])
    prob.solve()
    #print("Solution result is: %s" % prob.solution.get_status_string())
    #print(prob.solution.get_values())
    vertexCovers = []
    solutions = prob.solution.get_values()
    for j in range(0,len(solutions)):
        if solutions[j] == 1:
            vertexCovers.append(vertices[j])
    return vertexCovers

ModuleNotFoundError: No module named 'cplex'

In [13]:
graph5 = {
    "a": ["b", "c", "d", "e"],
    "d": ["e"],
    "c": ["b", "e"]
}
#       C           D
#      / \____    /  \
#     /   \   \__/    \
#    /     \    /\_____\
#   /       \  /       \\
# B----------A-----------E

def getNodes(graph):
    nodes = []
    for n in graph.keys():
        nodes.append(n)
    for n2 in graph.values():
        if type(n2) == "string":
            nodes.append(n2)
        else:
            nodes += n2
    return np.unique(nodes).tolist()

def getEdgesPair(graph, isDirected):
    edges = []
    for item in graph.items():
        for destination in item[1]:
            if (not isDirected):
                string1 = item[0] if item[0] < destination else destination
                string2 = destination if item[0] < destination else item[0]
            else:
                string1 = item[0]
                string2 = destination
            edges.append((string1,string2))
    return edges

vertex_cover(getNodes(graph5),getEdgesPair(graph5, False))

[[['a', 'b'], [1, 1]], [['a', 'c'], [1, 1]], [['a', 'd'], [1, 1]], [['a', 'e'], [1, 1]], [['d', 'e'], [1, 1]], [['b', 'c'], [1, 1]], [['c', 'e'], [1, 1]]]
['ab', 'ac', 'ad', 'ae', 'de', 'bc', 'ce']
[1, 1, 1, 1, 1, 1, 1]
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 5.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
MIP Presolve modified 3 coefficients.
Reduced MIP has 3 rows, 5 columns, and 9 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 3 rows, 5 columns, and 9 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 5.
MIP emphasis: balance optimality and feas

['a', 'b', 'e']